In [ ]:
import tensorflow as tf

# For showing plots inline
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
# change working directory to base, to make all imports and file paths work
import os
os.chdir(os.pardir)
print("Current directory: %s" % os.getcwd())

In [ ]:
import trainer.data_pipeline as dp
import trainer.constants as cst
from trainer.helpers import print_dict_keys, simple_plotly
from trainer.evaluation import get_predictions_results, plot_predictions_and_errors, plot_errors_and_counts
from server.clippy import Clippy, clipped_relu

In [ ]:
# make sure that saved_model has been created with the same version of TensorFlow
tf.__version__

In [ ]:
# Loading local model
model_dir = 'saved_model/clippy_1000'#"data/ion_age_20190614_205447/saved_model"
model = tf.keras.experimental.load_from_saved_model(model_dir, custom_objects={'clippy': Clippy(clipped_relu)})

scaling_factors = dp.load_scaling_factors()

In [ ]:
# Dataset for creating the results dataframe
dataset_dir = cst.SECONDARY_TEST_SET
window_size = 20
shift = 5
stride = 1
batch_size = 32

dataset = dp.create_dataset(dataset_dir,
                            window_size=window_size,
                            shift=shift,  # Can vary during validation
                            stride=stride,
                            batch_size=batch_size,  # Can vary during validation
                            cycle_length=1,  # To match original order (so no files get interleaved)
                            num_parallel_calls=1,  # Has to be equal or below cycle_length
                            shuffle=False,  # To match original order
                            repeat=False)

In [ ]:
results = get_predictions_results(model, dataset, scaling_factors)

In [ ]:
results.head(100)

In [ ]:
plot_predictions_and_errors(results, return_div=False)

In [ ]:
plot_errors_and_counts(results, window_size, inline=True)

In [ ]:
plot_errors_and_counts(results, window_size, height=500, show_count=False, cycle_bin_width=100, inline=True)